In [2]:
import pandas as pd
df1 = pd.read_csv('./1Food_Inspections.csv')
df2 = df1.loc[0:5000,].copy()

In [3]:


# Función que manda a llamar las trasnformaciones de estandarización de nombre
# de variables, ajustes en valores de varia
def cleaning(df):
    df = c_standarize_column_names(df)
    df = c_change_to_lowercase(df, ['aka_name', 'facility_type', 'risk', 'city', 'state', 'inspection_type',
                               'results', 'violations'])
    df = c_numeric_transformation_risk(df)
    df = c_categoric_transformation(df, ['aka_name', 'facility_type', 'inspection_type'])
    df = c_date_transformation('inspection_date', df)
    df = c_correct_chicago_in_state(df)
    df = c_drop_rows_not_chicago(df)
    df = c_drop_rows_not_illinois(df)
    df = c_drop_rows_nulls_0_in_license(df)
    df = c_drop_columns(df)
    return df


# Ajusta los nombres de las columnas del dataframe que se pase
# como parámetro (entre ellos cambia todo a minísuculas)
def c_standarize_column_names(df):
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.replace('#', '')
    df.columns = df.columns.str.lower()
    return df


# Ajusta los valores de los variables (columnas) que se pasen como
# parámetro del dataframe señalado
def c_change_to_lowercase(df, variables):
    for name in variables:
        df[name] = df[name].str.lower()
    return df


# Cambiamos la variable categorica de risk a numerica, considerando
# que existe un orden natural
# Obs: Para la categoría 'All' se le asignó el riesgo máximo de 1
#      pero no se tiene un fundamento. En la base se observa que
#      la mayoría de estos registros tienen por resultado de auditoría
#      ausente, por lo que más bien se trata de un sentinel value
def c_numeric_transformation_risk(df):
    df['risk'] = df['risk'].replace({'risk 1 (high)': 3, 'risk 2 (medium)': 2, 'risk 3 (low)': 1, 'all': 3})
    df['risk'] = df['risk'].convert_dtypes()
    return df


# Cambia el tipo de columna indicada como 'category'
def c_categoric_transformation(df, variables):
    for name in variables:
        df[name] = df[name].astype("category")
    return df


# Cambia el tipo de columna indicada como 'date'
def c_date_transformation(col, df):
    df[col] = pd.to_datetime(df[col])
    return df


# Homologa el valor de la variable
def c_correct_chicago_in_state(df):
    df['state'] = df['state'].replace(to_replace=["312chicago", "cchicago", "chchicago", "chcicago", \
                                           "chicagochicago", "chicagohicago", "chicagoi", \
                                           "chicago."], value="chicago")
    return df


# Función para quitar los registros que no corresponden a chicago
def c_drop_rows_not_chicago(df):
    df = df[df['city'] == 'chicago']
    return df

def c_drop_rows_not_illinois(df):
    df = df[df['state'] == 'il']
    return df


# Función para quitar los registros cuyo numero de licencia es cero o
# nulo (vacío)
def c_drop_rows_nulls_0_in_license(df):
    df = df[df['license_'].notnull()]
    df = df[df['license_'] != 0]
    return df


# Se seleccionan las variables deseadas
# Obs: Quizá aka_name podría servir, ya que contiene info del tipo de restaurante
#      Para pruebas unitarias se podría llenar el zip con los valores de latitud y longitud
#      y/o validar la congruencia entre estos 2 datos
def c_drop_columns(df):
    # df = df.drop(['dba_name', 'aka_name', 'address', 'city', 'state', 'location'], axis=1)
    df = df.drop(['city', 'state'], axis=1)
    return df

In [4]:
import pandas as pd
from datetime import datetime
from datetime import date
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
import numpy as np
#from src.etl.cleaning import c_date_transformation

# Función que manda a llamar las funciones con las que se crean nuevas
# variables
def feature_engineering(df):
    df = c_date_transformation('inspection_date', df)
    fe_add_column_year_of_insp_date(df)
    fe_add_column_month_of_insp_date(df)
    fe_add_column_day_of_insp_date(df)
    fe_add_column_pass_int(df)
    #df = fe_add_column_days_since_last_insp(df)
    df = fe_add_column_approved_insp(df)
    df = fe_add_column_num_viol_last_insp(df)
    df = fe_imputer(df)
    df = dummies_day(df)
    df = dummies_month(df)
    fe_add_column_month_of_insp_date(df)
    return df


# Se crea variable con el numero de mes en que se hizo la inspección
def fe_add_column_year_of_insp_date(df):
    df['inspection_year'] = df['inspection_date'].dt.year

# Se crea variable con el numero de mes en que se hizo la inspección
def fe_add_column_month_of_insp_date(df):
    df['inspection_month'] = df['inspection_date'].dt.month


# Se crea variable con el numero de semana en que se hizo la inspección
def fe_add_column_day_of_insp_date(df):
    df['inspection_day'] = df['inspection_date'].dt.dayofweek


# Función que crea la variable objetivo que señala con 1 si se pasó la
# inspección y 0 en caso contrario
# Obs: Se debe considerar 'not ready', 'out of business' o
#      'business not located' como 'fail'?
def fe_add_column_pass_int(df):
    dic_results = {'pass': 1,
                   'out of business': 0,
                   'no entry': 0,
                   'fail': 0,
                   'not ready': 0,
                   'pass w/ conditions': 1,
                   'business not located': 0}
    df['pass'] = df.results.map(dic_results)
    df['pass'] = df['pass'].convert_dtypes()


# Función que crea una variable que señala los días transcurridos
# desde la última inspección (requiere que todos los registros
# tengan un numero de licencia)
# Obs: Toma unos 2.5 minutos en ejecutar
def fe_add_column_days_since_last_insp(df):
    # Se ordena en forma ascendente por license_ (que se considera representa un mismo individuo)
    # y en forma descendente por fecha de inspección, lo cual será de apoyo para recorrer todos los registros
    # y hacer cálculos de nuevas variables
    df = df.sort_values(['license_', 'inspection_date'], ascending=[True, False])
    df = df.reset_index(drop=True)
    df['days_since_last_inspection'] = 0
    flag = 0
    for i in range(df.shape[0]):
        if df.iloc[i]['license_'] != flag:
            # Cada que observamos un nuevo numero de licencia, estaremos parados en la última inspección
            # por lo que calcularemos los días transcurridos del día en que se ejecute este código y la fecha
            # de dicha inspección
            df.at[i, 'days_since_last_inspection'] = (datetime.now() - df.iloc[i]['inspection_date']).days
        else:
            # Cuando estamos en esta parte del operador if, estamos en una 2da, 3ra, etc, observación de un mismo
            # numero de licencia
            df.at[i, 'days_since_last_inspection'] = (
                        df.iloc[i - 1]['inspection_date'] - df.iloc[i]['inspection_date']).days
        flag = df.iloc[i]['license_']
    return df


# Función que crea una variable que señala el número de inspecciones anteriores
# Obs: Toma unos 2.5 minutos en ejecutar
def fe_add_column_approved_insp(df):
    df = df.sort_values(['license_', 'inspection_date'], ascending=[True, True])
    df = df.reset_index(drop=True)
    df['approved_insp'] = 0
    flag = 0
    for i in range(df.shape[0]):
        if df.iloc[i]['license_'] != flag:
            df.at[i, 'approved_insp'] = 0
        else:
            # El conteo de inspecciones pasadas no hace diferencia entre las que fueron aprobadas
            # con condiciones o sin condiciones
            if df.iloc[i - 1]['results'] == 'pass w/ conditions' or df.iloc[i - 1]['results'] == 'pass':
                # si la última inspección fue aprobada se suma 1 al conteo acumulado
                df.at[i, 'approved_insp'] = df.iloc[i - 1]['approved_insp'] + 1
            else:
                # si la última inspección fue reaprobada no se suma nada al conteo acumulado
                df.at[i, 'approved_insp'] = df.iloc[i - 1]['approved_insp']
        flag = df.iloc[i]['license_']
    return df


def fe_add_column_num_viol_last_insp(df):
    df = df.sort_values(['license_', 'inspection_date'], ascending=[True, True])
    df = df.reset_index(drop=True)
    df['num_viol_last_insp'] = 0
    flag = 0
    for i in range(df.shape[0]):
        if df.iloc[i]['license_'] != flag:
            df.at[i, 'num_viol_last_insp'] = 0
        else:
            if pd.isnull(df.iloc[i - 1]['violations']):
                df.at[i, 'num_viol_last_insp'] = 0
            else:
                # si el valor de violation de la inspección es no nullo es porque existe una violación (por ello
                # de entrada se suma 1), cada violación adicional está señalado separandolo con '| ' (por ello
                # se cuentan las veces que aparece dicha secuencia). La violación 60 se refiere a comentarios relativos
                # al complimiento de violaciones anteriores ('previous core violation corrected') y como no es una violación
                # adicional a la inspección, si se tiene esta nota se resta 1
                df.at[i, 'num_viol_last_insp'] = 1 + df.iloc[i - 1]['violations'].count('| ') - df.iloc[i - 1][
                    'violations'].count('previous core violation corrected')
        flag = df.iloc[i]['license_']
    return df


def fe_imputer(df):
    # Función para imputar columnas específicas
    # Obs: Cada variable señalada se imputa con el valor más frecuente. Dada la frecuencia observada
    #      en los datos, se puede decir que:
    #      - los campos vacíos de 'dba_name' se llenas con 'subway'
    #      - los campos vacíos de 'aka_name' se llenas con 'subway'
    #      - los campos vacíos de 'facility_type' se llenas con 'restaurant'
    #      - los campos vacíos de 'risky' se llenas con 1
    #      - El valor que se inserte en los campos vacíos de 'zip' podría no se congruente con 'location'
    #        (la mayoría de las veces que no se tiene zip sí se tiene 'location', pero en general son pocos
    #        datos faltantes de 'zip')
    #      - los campos vacíos de 'inspection_type' se llenas con 'canvass'
    #      - los campos vacíos de 'results' se llenas con 'pass'
    #      En general no hay datos faltantes en 'inspection_id', 'risk', 'inspection_date', 'results'
    transformers = [
        ('impute_facility_type', SimpleImputer(strategy="most_frequent"), ['facility_type']),
        ('impute_risk', SimpleImputer(strategy="most_frequent"), ['risk']),
        ('impute_zip', SimpleImputer(strategy="most_frequent"), ['zip']),
        ('impute_inspection_type', SimpleImputer(strategy="most_frequent"), ['inspection_type']),
        ('impute_results', SimpleImputer(strategy="most_frequent"), ['results']),
        ('impute_latitude', SimpleImputer(strategy="most_frequent"), ['latitude']),
        ('impute_longitude', SimpleImputer(strategy="most_frequent"), ['longitude'])
    ]
    # Definimos el transformador con las transformaciones arriba definidas
    col_trans = ColumnTransformer(transformers, remainder="passthrough", n_jobs=-1, verbose=True)
    # Ajustamos
    col_trans.fit(df)
    # Obtenemos el resultado de las transformaciones (imputaciones) aplicadas a la base
    aux = col_trans.transform(df)
    # Generamos un arreglo auxiliar que contiene los nombres de las variables transformadas
    aux_var_imput = pd.DataFrame(transformers[:])[0].str.replace('impute_', '')
    # Generamos un arreglo auxiliar que contiene las variables no transformadas
    aux_var_no_imput = df.columns[~np.in1d(df.columns, aux_var_imput)]
    # Guardamos el orden original de las columnas de la base
    col_original_order = df.columns
    # creamos un dataframe con los resultados del transformador, plasmando los
    # de columnas correspondientes

    # creamos un dataframe con las variables dummies y otro dataframe con las variables
    # que no transformamos (no dummies) para después unirlos
    aux_df_var_no_imput = df[df.columns[~np.in1d(df.columns, aux_var_imput)]]
    aux_df_var_imput = pd.DataFrame(aux[:, 0:len(aux_var_imput)], columns=aux_var_imput)
    # unimos los 2 dataframes para tener una única base conservando el tipo
    # de datos que previamente definimos para las variables que no transformamos
    df = pd.concat([aux_df_var_no_imput, aux_df_var_imput], axis=1)
    df = df[col_original_order]
    return df


# def fe_dummier(df):
#     transformer = [('one_hot', OneHotEncoder(), ['inspection_month', 'inspection_day'])]
#     col_trans = ColumnTransformer(transformer, remainder="passthrough", n_jobs=-1, verbose=True)
#     col_trans.fit(df)
#     aux = col_trans.transform(df)
#     aux_var_dummies = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', \
#                        'mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
#     # identificamos las variables que no son dummies
#     aux_var_no_dummies = df.columns[~np.in1d(df.columns, ['inspection_month', 'inspection_day'])]

#     # creamos un dataframe con las variables dummies y otro dataframe con las variables
#     # que no transformamos (no dummies) para después unirlos
#     aux_df_var_no_dummies = df[df.columns[~np.in1d(df.columns, ['inspection_month', 'inspection_day'])]]
#     aux_df_var_dummies = pd.DataFrame(aux[:, 0:len(aux_var_dummies)], columns=aux_var_dummies).convert_dtypes()
#     # unimos los 2 dataframes para tener una única base conservando el tipo
#     # de datos que previamente definimos para las variables que no transformamos
#     df = pd.concat([aux_df_var_no_dummies, aux_df_var_dummies], axis=1)
#     return df

def dummies_month(df):
    d = {
        1: 'ene',
        2: 'feb',
        3: 'mar',
        4: 'abr',
        5: 'may',
        6: 'jun',
        7: 'jul',
        8: 'ago',
        9: 'sep',
        10: 'oct',
        11: 'nov',
        12: 'dic'
    }
    # Dataframe de ceros con las columnas de todos los meses
    df_total_dummies = pd.DataFrame(0, index=np.arange(df.shape[0]), columns=d.values())
    # Dataframe con las dummies creadas a partir de los valores encontrados
    # (no necesariamente estarían todos los meses)
    df_current_dummies = pd.get_dummies(df['inspection_month'])
    # Ponemos el nombre de estas dummies acorde al diccionario
    # (ejemplo: la columan 1 cambia de nombre a 'ene')
    df_current_dummies.columns = df_current_dummies.columns.map(d)
    # dataframe auxiliar que une los dos dataframes
    aux = pd.concat([df_current_dummies, df_total_dummies], axis=1)
    # quitamos las columnas duplicadas, prevaleciendo las que primero encuentra
    # que son las current dummies
    df_dummies = aux.loc[:,~aux.columns.duplicated()]
    df_dummies = df_dummies.astype(int)
    # se regresa el dataframe original sin la variable a la que se crearon las dummies,
    # uniando al final las columnas dummies
    df_new = pd.concat([df.drop(['inspection_month'], axis=1), df_dummies[d.values()]], axis=1)
    return df_new

def dummies_day(df):
    d = {
        1: 'mon',
        2: 'tue',
        3: 'wed',
        4: 'thu',
        5: 'fri',
        6: 'sat',
        7: 'sun'
    }
    # Dataframe de ceros con las columnas de todos los dias de la semana
    df_total_dummies = pd.DataFrame(0, index=np.arange(df.shape[0]), columns=d.values())
    # Dataframe con las dummies creadas a partir de los valores encontrados
    # (no necesariamente estarían todos los dias)
    df_current_dummies = pd.get_dummies(df['inspection_day'])
    # Ponemos el nombre de estas dummies acorde al diccionario
    # (ejemplo: la columan 1 cambia de nombre a 'mon')
    df_current_dummies.columns = df_current_dummies.columns.map(d)
    # dataframe auxiliar que une los dos dataframes
    aux = pd.concat([df_current_dummies, df_total_dummies], axis=1)
    # quitamos las columnas duplicadas, prevaleciendo las que primero encuentra
    # que son las current dummies
    df_dummies = aux.loc[:,~aux.columns.duplicated()]
    df_dummies = df_dummies.astype(int)
    # se regresa el dataframe original sin la variable a la que se crearon las dummies,
    # uniando al final las columnas dummies
    df_new = pd.concat([df.drop(['inspection_day'], axis=1), df_dummies[d.values()]], axis=1)
    return df_new

In [5]:
df3 = cleaning(df2)

In [6]:
def split_tiempo(archivo,campo_criterio,criterio):
    datos_ordenados=archivo.sort_values(by=campo_criterio)
    archivo_2=datos_ordenados.loc[datos_ordenados[campo_criterio]<=criterio]
    archivo_3=datos_ordenados.loc[datos_ordenados[campo_criterio]>criterio]
    return archivo_2, archivo_3


In [7]:
#df3.sort_values('inspection_date', inplace = True)
train, test = split_tiempo(df3, 'inspection_date', '2020-12-31')

In [8]:

fe_train = feature_engineering(train)

In [9]:
X = fe_train.drop(['dba_name', 'aka_name', 'facility_type', 'address', 'inspection_date', 'inspection_type', 'violations', 'results', 'location', 'pass'], axis=1)
y = fe_train ['pass']

In [18]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [19]:
tscv = TimeSeriesSplit(n_splits=5)

cv=[(train_index, test_index)
      for train_index,test_index in tscv.split(X)]


In [26]:
def magic_loop(X_train,y_train):
    #for i in range(0,len(modelos_to_run)):
    classifier = RandomForestClassifier()
    hyper_param_grid= {'n_estimators': [100,500,800], 
                    'max_depth': [1,5,10,50], 
                    'max_features': ['sqrt','log2'],
                    'min_samples_split': [2,5,10],
                       'min_samples_leaf':[1,2,4]}
    grid_search1 = GridSearchCV(classifier, 
                           hyper_param_grid, 
                           scoring = 'f1',
                           cv = cv, 
                           n_jobs = -1,
                           verbose = 3)
    grid_search1.fit(X_train, y_train)
    cv_results = pd.DataFrame(grid_search1.cv_results_)
    results_1 = cv_results.sort_values(by='rank_test_score', ascending=True)
       
        
    classifier = tree.DecisionTreeClassifier()
    hyper_param_grid= {'max_depth': [3,5,10,25],
                       'min_samples_split': [2,5,10,15],
                       'min_samples_leaf':[1,2,4],
                       'max_features': ['sqrt','log2']}
    grid_search2 = GridSearchCV(classifier, 
                           hyper_param_grid, 
                           scoring = 'f1',
                           cv = cv, 
                           n_jobs = -1,
                           verbose = 3)
    grid_search2.fit(X_train, y_train)
    cv_results = pd.DataFrame(grid_search2.cv_results_)
    results_2 = cv_results.sort_values(by='rank_test_score', ascending=True)

    return results_1,results_2
    


In [27]:
results_1,results_2 =magic_loop(X, y.astype(int))

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Fitting 5 folds for each of 96 candidates, totalling 480 fits


In [28]:
results_1.head(1)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
195,0.534688,0.172015,0.03422,0.000976,50,log2,1,10,100,"{'max_depth': 50, 'max_features': 'log2', 'min...",0.438406,0.463918,0.414557,0.529703,0.432836,0.455884,0.040146,1


In [29]:
results_2.head(1)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
72,0.015742,0.004996,0.006013,0.000955,25,sqrt,1,2,"{'max_depth': 25, 'max_features': 'sqrt', 'min...",0.419872,0.512376,0.577595,0.458716,0.489747,0.491661,0.053022,1


In [30]:
import time 


from sklearn.model_selection import train_test_split


# ocuparemos un RF
classifier = RandomForestClassifier(oob_score=True, random_state=1234)
# separando en train, test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# definicion de los hiperparametros que queremos probar
hyper_param_grid = {'n_estimators': [100, 500, 800, 1000], 
                    'max_depth': [1, 5, 10, 20, 50],
                    'min_samples_split': [2, 5, 10]}

# ocupemos grid search!
gs = GridSearchCV(classifier, 
                           hyper_param_grid, 
                           scoring = 'precision',
                           cv = 5, 
                           n_jobs = -1)
start_time = time.time()
gs.fit(X, y.astype(int))
print("Tiempo en ejecutar: ", time.time() - start_time)

Tiempo en ejecutar:  176.01030468940735


In [31]:
gs.best_params_

{'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100}

In [32]:
gs.best_score_

0.5900753691036479

In [33]:
gs.best_estimator_

RandomForestClassifier(max_depth=5, min_samples_split=5, oob_score=True,
                       random_state=1234)

In [34]:
gs.best_estimator_.oob_score_

0.6070702966273872

In [ ]:
days_encoder = OneHotEncoder(categories = 'auto')
days_encoder.fit(train.inspection_day.values.reshape(train.shape[0],1))
days_encoder.transform(train.inspection_day.values.reshape(train.shape[0],1))
days_encoder.categories_

In [ ]:
month_encoder = OneHotEncoder(categories = 'auto')
month_encoder.fit(train.inspection_month.values.reshape(train.shape[0],1))
month_encoder.transform(train.inspection_month.values.reshape(train.shape[0],1))
month_encoder.categories_

In [ ]:
#train
#X = train.values.reshape(train.shape[0],19)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y.astype(int), random_state=4)

In [ ]:
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train, y_train)

In [ ]:
print("Accuracy on training set: {:.3f}".format(gs.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(gs.score(X_test, y_test)))

In [ ]:
var_select = df4[['inspection_id', 'license_', 'risk', 'zip', 'latitude', 'longitude',
       'inspection_year', 'approved_insp', 'num_viol_last_insp', 'mon', 'tue',
       'wed', 'thu', 'fri', 'sat', 'sun', 'ene', 'feb', 'mar', 'abr', 'may',
       'jun', 'jul', 'ago', 'sep', 'oct', 'nov', 'dic', 'inspection_month', 'pass']]

In [ ]:
#X1_train, X1_test, y1_train, y1_test = split_aleatorio(var_in, var_out, 80,4)
X_train, X_test, y_train, y_test = split_tiempo(var_select,'inspection_year',2020)

In [ ]:
#var_in = df4.drop(['dba_name', 'aka_name', 'facility_type', 'address', 'inspection_date', 'inspection_type', 'violations', 'results', 'location', 'pass'], axis=1)

In [ ]:
#var_in = df.drop(['pass'], axis=1)
#var_out = df['pass']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.feature_selection import VarianceThreshold

variance_threshold = VarianceThreshold(threshold=0.1)
variance_threshold.fit(var_in)

variance_threshold.transform(var_in)

variance_threshold.variances_.shape

var_in.columns



## Model selection

In [ ]:
X = X_train
#y = pd.DataFrame(y_train)
y = y_train
print(X.shape, y.shape)

In [ ]:


np.random.seed(20201117)



In [ ]:
from sklearn.tree import DecisionTreeClassifier


#classifier = RandomForestClassifier(oob_score=True, random_state=1234)
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X, y)

In [ ]:
y_test.unique()

In [ ]:
print("Accuracy on training set: {:.3f}".format(tree.score(X, y)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, pd.DataFrame(y_test))))

In [ ]:


import time 

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# ocuparemos un RF
classifier = RandomForestClassifier(oob_score=True, random_state=1234)
# separando en train, test
X_train, X_test, y_train, y_test = train_test_split(X, y)

# definicion de los hiperparametros que queremos probar
hyper_param_grid = {'n_estimators': [100, 500, 800, 1000], 
                    'max_depth': [1, 5, 10, 20, 50],
                    'min_samples_split': [2, 5, 10]}

# ocupemos grid search!
gs = GridSearchCV(classifier, 
                           hyper_param_grid, 
                           scoring = 'precision',
                           cv = 5, 
                           n_jobs = -1)
start_time = time.time()
gs.fit(X, y)
print("Tiempo en ejecutar: ", time.time() - start_time)



In [ ]:
def RandomForest(train_variables_input,test_variables_input,train_variable_output,numero_arboles,umbral,semilla):
    #One hot encoder
    #lista_variables_categoricas=train_variables_input.select_dtypes(include = 'object').columns.values
    #train_variables_input=pd.get_dummies(train_variables_input,columns=lista_variables_categoricas,prefix=lista_variables_categoricas)
    #test_variables_input=pd.get_dummies(test_variables_input,columns=lista_variables_categoricas,prefix=lista_variables_categoricas)
    
    nombres_columnas=train_variables_input.columns.values
    variables_input_array=train_variables_input.values.reshape(train_variables_input.shape[0],train_variables_input.shape[1])
    clf = RandomForestClassifier(n_estimators=numero_arboles, random_state=semilla, n_jobs=-1)
    #sfm = SelectFromModel(clf, threshold=umbral)
    clf.fit(variables_input_array, train_variable_output)
    feature_importances = pd.DataFrame(clf.feature_importances_,
                                   index = train_variables_input.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
    feature_importances=feature_importances.loc[feature_importances['importance']>=umbral]
    variables_seleccionadas=list(feature_importances.index)
    train_variables_seleccionadas=train_variables_input.loc[:,variables_seleccionadas]
    test_variables_seleccionadas=test_variables_input.loc[:,variables_seleccionadas]
    return feature_importances,train_variables_seleccionadas,test_variables_seleccionadas

In [ ]:
RandomForest(X_train, X_test, y_train, 500, .05, 0)

In [ ]:
importancia_features,X_train,X_test=feature_selection.RandomForest(X_train,X_test,y_train,500,.05,0)

In [ ]:
dft = df.sort_values(by=['inspection_year','inspection_month'],ascending=True)
#Dividir la base en train y test
df_train,df_test = train_test_split(df,test_size=0.2,random_state=1234,shuffle=True)

df_input_vars = df_train.drop(['pass'], axis=1)

X = df_input_vars
y = df_train['pass'].values

classifier = RandomForestClassifier(oob_score=True, random_state=1234)

# separando en train, test
X_train, X_test, y_train, y_test = train_test_split(X, y)
# definicion de los hiperparametros que queremos probar
hyper_param_grid = {'n_estimators': [10,20], 
                    'max_depth': [1, 2, 5],
                    'min_samples_split': [2, 4]}

#Time Series cross-validator
tscv = TimeSeriesSplit(n_splits=5)
gs = GridSearchCV(classifier, 
                           hyper_param_grid, 
                           scoring = 'precision',
                           cv = tscv,
                            
                           n_jobs = -1)
gs.fit(X_train, y_train) #AQUÍ ESTÁ EL ERROR

#Importancia con el mejor modelo
importance = gs.best_estimator_.feature_importances_
dataset_2 = pd.DataFrame({'importance': importance, 'col_name': names}).sort_values(by='importance',ascending=False)
dataset_3 = dataset_2[dataset_2['importance']>=0.07]
columnas_modelo = dataset_3['col_name'].values
df = pd.DataFrame(X_train.toarray())
df.columns = names
df_col = df[columnas_modelo]
label = pd.DataFrame(y_train,columns=['label'])
df_modelo = pd.concat([df_col,label],axis=1)        
df_modelo,df_test

In [ ]:
y_train.unique()

In [ ]:


#pd.DataFrame(columns=['X','y'])
sr = pd.concat([X,y], axis =1)
split_tiempo2(sr,)
